In [1]:
import json

from rauth import OAuth1Service, OAuth2Service
from flask import current_app, url_for, request, redirect, session


class OAuthSignIn(object):
    providers = None

    def __init__(self, provider_name):
        self.provider_name = provider_name
        credentials = current_app.config['OAUTH_CREDENTIALS'][provider_name]
        self.consumer_id = credentials['id']
        self.consumer_secret = credentials['secret']

    def authorize(self):
        pass

    def callback(self):
        pass

    def get_callback_url(self):
        return url_for('oauth_callback', provider=self.provider_name,
                       _external=True)

    @classmethod
    def get_provider(self, provider_name):
        if self.providers is None:
            self.providers = {}
            for provider_class in self.__subclasses__():
                provider = provider_class()
                self.providers[provider.provider_name] = provider
        return self.providers[provider_name]


class VkSignIn(OAuthSignIn):
    def __init__(self):
        super(VkSignIn, self).__init__('vk')
        self.service = OAuth2Service(
            name='vk',
            client_id=self.consumer_id,
            client_secret=self.consumer_secret,
            authorize_url='https://oauth.vk.com/authorize',
            access_token_url='https://oauth.vk.com/access_token',
            base_url = None
        )

    def authorize(self):
        return redirect(self.service.get_authorize_url(
            scope='friends, email, offline',
            response_type='code',
            redirect_uri = self.get_callback_url())
        )

    def callback(self):
        if 'code' not in request.args:
            return None, None
        oauth_session = self.service.get_raw_access_token(
            data = {'code': request.args['code'],
                  'client_id': self.consumer_id, 
                  'client_secret' :self.consumer_secret,
                  'redirect_uri': self.get_callback_url()},
        )
        
        user_id = oauth_session.json().get('user_id')
        token = oauth_session.json().get('access_token')
        
        return user_id, token

In [48]:


from flask import Flask, redirect, url_for, render_template, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, logout_user,\
    current_user

app = Flask(__name__)

app.config['SECRET_KEY'] = 'top secret!'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite'
app.config['OAUTH_CREDENTIALS'] = {
    'vk': {
        'id': '7576229',
        'secret': '5gUBRqwNesbiF6og9Eq5'
    },
}

db = SQLAlchemy(app)
lm = LoginManager(app)
lm.login_view = 'index'

class User(UserMixin, db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key = True)
    user_id = db.Column(db.String(64), nullable = False, unique = True)
    username = db.Column(db.String(128), nullable = False, unique = True)
    token = db.Column(db.String(256), nullable = False, unique = True)
    
@lm.user_loader
def load_user(id):
    return User.query.get(int(id))


@app.route('/')
def index():
    if current_user.is_authenticated:
        friends_list = get_friends_curr_user(current_user.token, current_user.user_id)
        return render_template('index.html', friends_list = friends_list)
    return render_template('index.html')


@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))


@app.route('/authorize/<provider>')
def oauth_authorize(provider):
    if not current_user.is_anonymous:
        return redirect(url_for('index'))
    oauth = OAuthSignIn.get_provider(provider)
    return oauth.authorize()


@app.route('/callback/<provider>')
def oauth_callback(provider):
    if not current_user.is_anonymous:
        return redirect(url_for('index'))
    oauth = OAuthSignIn.get_provider(provider)
    user_id, token = oauth.callback()
    
    if user_id is None:
        flash('Authentication failed.')
        return redirect(url_for('index'))
    
    user = User.query.filter_by(user_id = user_id).first()
    
    if not user:
        username = get_concate_username(token, user_id)
        
        user = User(user_id = user_id, token = token, username = username)
        db.session.add(user)
        db.session.commit()
        
    login_user(user, True)
    return redirect(url_for('index'))

def get_concate_username(token, user_id):
    data = {'user_id': user_id,
            'v': '5.92', 
            'access_token': token,
           }
    
    user_info = requests.post('https://api.vk.com/method/users.get', data = data)
    
    if 'response' in user_info.json():
        user_info = user_info.json().get('response')[0]
        first_name, last_name = user_info['first_name'], user_info['last_name']
        return first_name + ' ' + last_name
    return None

def get_friends_curr_user(token, user_id):
    tmp = {}
    data = {'user_id': user_id,
            'v': '5.92', 
            'access_token': token,
            'count': 5
           }
    
    user_list = requests.post('https://api.vk.com/method/friends.get', data = data)
    if 'response' in user_list.json():
        user_list = user_list.json().get('response').get('items')
        for user_id in user_list:
            tmp[user_id] = {
                'user_url': f'https://vk.com/id{user_id}',
                'user_name': get_concate_username(token, user_id)
            }
    return tmp
    
if __name__ == '__main__':
    db.create_all()
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/home/pi/flask_test/venv/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2020 09:57:49] "GET /authorize/vk HTTP/1.1" 302 -
127.0.0.1 - - [25/Aug/2020 09:57:54] "GET / HTTP/1.1" 200 -
